In [1]:
#Import packages
import numpy as np
import pandas as pd
import torch

In [2]:
#Set seed
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Model 

In [3]:
from torch import nn

In [4]:
class Binary_SVM(nn.Module):
    def __init__(self, n_features):
        super(Binary_SVM, self).__init__()
        self.linear = nn.Linear(n_features, 1)
    def forward(self, x):
        out = self.linear(x) # (N x 1)
        out = out.squeeze() #(N)
        return out

# Get Breast Cancer Dataset

In [5]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

In [6]:
#Create pandas dataframe

df = pd.DataFrame(data["data"],columns = data["feature_names"])
df['target'] = data['target']
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
#Class Name
data.target_names

array(['malignant', 'benign'], dtype='<U9')

In [8]:
#Create train-test-split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =train_test_split(data['data'], 2 * data['target'] -1,test_size= 0.2, random_state = seed )

In [9]:
#Create tensor dataset and dataloader
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader

train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
test_dataset= TensorDataset(torch.tensor(x_test), torch.tensor(y_test))

train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size =32, shuffle= False)

# Model Training

In [10]:
#Create Model

model = Binary_SVM(x_train.shape[1])

In [15]:
#Hyperparameters and Optimizer
epochs = 1000
C = 1.0
lr = 0.001

optimizer = torch.optim.Adam(model.parameters(),lr = lr)

In [16]:
#Model Training

total_loss = 0
n_batch =0

model.train()
for epoch in range(epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        x_train, y_train =batch
        output = model(x_train.float())
        loss = 0.5 * torch.norm(model.linear.weight.squeeze()) ** 2
        loss += C * torch.clamp(1- y_train * output,min = 0).mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        n_batch += 1
        
    print(f"Epoch : {epoch}\t Loss : {total_loss / n_batch}")   

Epoch : 0	 Loss : 0.278997540473938
Epoch : 1	 Loss : 0.28406858444213867
Epoch : 2	 Loss : 0.2715037763118744
Epoch : 3	 Loss : 0.2639671266078949
Epoch : 4	 Loss : 0.27572014927864075
Epoch : 5	 Loss : 0.2733612358570099
Epoch : 6	 Loss : 0.2775592505931854
Epoch : 7	 Loss : 0.28036683797836304
Epoch : 8	 Loss : 0.27643319964408875
Epoch : 9	 Loss : 0.2736806869506836
Epoch : 10	 Loss : 0.27055612206459045
Epoch : 11	 Loss : 0.26869359612464905
Epoch : 12	 Loss : 0.26622143387794495
Epoch : 13	 Loss : 0.26533424854278564
Epoch : 14	 Loss : 0.26436614990234375
Epoch : 15	 Loss : 0.2625672221183777
Epoch : 16	 Loss : 0.2632630169391632
Epoch : 17	 Loss : 0.2621096074581146
Epoch : 18	 Loss : 0.2603335678577423
Epoch : 19	 Loss : 0.26129671931266785
Epoch : 20	 Loss : 0.26218563318252563
Epoch : 21	 Loss : 0.26038989424705505
Epoch : 22	 Loss : 0.26135748624801636
Epoch : 23	 Loss : 0.2593098282814026
Epoch : 24	 Loss : 0.2593553960323334
Epoch : 25	 Loss : 0.260841965675354
Epoch : 26	

Epoch : 217	 Loss : 0.20943550765514374
Epoch : 218	 Loss : 0.20939595997333527
Epoch : 219	 Loss : 0.20922604203224182
Epoch : 220	 Loss : 0.20902642607688904
Epoch : 221	 Loss : 0.2089705765247345
Epoch : 222	 Loss : 0.20873039960861206
Epoch : 223	 Loss : 0.2084929496049881
Epoch : 224	 Loss : 0.20833244919776917
Epoch : 225	 Loss : 0.20821478962898254
Epoch : 226	 Loss : 0.20801104605197906
Epoch : 227	 Loss : 0.2077987641096115
Epoch : 228	 Loss : 0.20758239924907684
Epoch : 229	 Loss : 0.207493394613266
Epoch : 230	 Loss : 0.20737092196941376
Epoch : 231	 Loss : 0.20735111832618713
Epoch : 232	 Loss : 0.20713426172733307
Epoch : 233	 Loss : 0.20688648521900177
Epoch : 234	 Loss : 0.2069505751132965
Epoch : 235	 Loss : 0.2069462537765503
Epoch : 236	 Loss : 0.20697785913944244
Epoch : 237	 Loss : 0.20710501074790955
Epoch : 238	 Loss : 0.20696057379245758
Epoch : 239	 Loss : 0.20700395107269287
Epoch : 240	 Loss : 0.20684920251369476
Epoch : 241	 Loss : 0.20697325468063354
Epoch :

Epoch : 426	 Loss : 0.18933789432048798
Epoch : 427	 Loss : 0.1892506629228592
Epoch : 428	 Loss : 0.18916364014148712
Epoch : 429	 Loss : 0.18908391892910004
Epoch : 430	 Loss : 0.18898117542266846
Epoch : 431	 Loss : 0.18892988562583923
Epoch : 432	 Loss : 0.1889338493347168
Epoch : 433	 Loss : 0.1890299916267395
Epoch : 434	 Loss : 0.18895797431468964
Epoch : 435	 Loss : 0.18885259330272675
Epoch : 436	 Loss : 0.18875622749328613
Epoch : 437	 Loss : 0.1886506825685501
Epoch : 438	 Loss : 0.18859611451625824
Epoch : 439	 Loss : 0.18848484754562378
Epoch : 440	 Loss : 0.18841440975666046
Epoch : 441	 Loss : 0.18831288814544678
Epoch : 442	 Loss : 0.18820790946483612
Epoch : 443	 Loss : 0.1880849301815033
Epoch : 444	 Loss : 0.18798205256462097
Epoch : 445	 Loss : 0.1878637671470642
Epoch : 446	 Loss : 0.1878775656223297
Epoch : 447	 Loss : 0.18794706463813782
Epoch : 448	 Loss : 0.1878661811351776
Epoch : 449	 Loss : 0.18775007128715515
Epoch : 450	 Loss : 0.18771147727966309
Epoch : 

Epoch : 637	 Loss : 0.18068107962608337
Epoch : 638	 Loss : 0.18064530193805695
Epoch : 639	 Loss : 0.18066371977329254
Epoch : 640	 Loss : 0.18060478568077087
Epoch : 641	 Loss : 0.18056970834732056
Epoch : 642	 Loss : 0.1805478185415268
Epoch : 643	 Loss : 0.1804877370595932
Epoch : 644	 Loss : 0.18046145141124725
Epoch : 645	 Loss : 0.18043120205402374
Epoch : 646	 Loss : 0.18041278421878815
Epoch : 647	 Loss : 0.1804439276456833
Epoch : 648	 Loss : 0.18042226135730743
Epoch : 649	 Loss : 0.18040385842323303
Epoch : 650	 Loss : 0.180456280708313
Epoch : 651	 Loss : 0.1805279552936554
Epoch : 652	 Loss : 0.18055394291877747
Epoch : 653	 Loss : 0.18055464327335358
Epoch : 654	 Loss : 0.1805432140827179
Epoch : 655	 Loss : 0.18052251636981964
Epoch : 656	 Loss : 0.1804589480161667
Epoch : 657	 Loss : 0.1804332584142685
Epoch : 658	 Loss : 0.1803624927997589
Epoch : 659	 Loss : 0.18037936091423035
Epoch : 660	 Loss : 0.18041670322418213
Epoch : 661	 Loss : 0.18038558959960938
Epoch : 66

Epoch : 849	 Loss : 0.17674724757671356
Epoch : 850	 Loss : 0.1767440289258957
Epoch : 851	 Loss : 0.17670327425003052
Epoch : 852	 Loss : 0.17670565843582153
Epoch : 853	 Loss : 0.17670048773288727
Epoch : 854	 Loss : 0.176714226603508
Epoch : 855	 Loss : 0.17667661607265472
Epoch : 856	 Loss : 0.1766502410173416
Epoch : 857	 Loss : 0.17662224173545837
Epoch : 858	 Loss : 0.17659856379032135
Epoch : 859	 Loss : 0.17658759653568268
Epoch : 860	 Loss : 0.17674198746681213
Epoch : 861	 Loss : 0.17688971757888794
Epoch : 862	 Loss : 0.1768670678138733
Epoch : 863	 Loss : 0.17685936391353607
Epoch : 864	 Loss : 0.176853209733963
Epoch : 865	 Loss : 0.17691431939601898
Epoch : 866	 Loss : 0.17688138782978058
Epoch : 867	 Loss : 0.17686942219734192
Epoch : 868	 Loss : 0.1768529713153839
Epoch : 869	 Loss : 0.1768105924129486
Epoch : 870	 Loss : 0.1768011599779129
Epoch : 871	 Loss : 0.17675167322158813
Epoch : 872	 Loss : 0.1767248660326004
Epoch : 873	 Loss : 0.1766829490661621
Epoch : 874	

# Evaluation

In [17]:
def accuracy(true_y, predicted_y):
    return (true_y == predicted_y).mean()

In [18]:
total_loss = 0
n_batch = 0
true_y = np.array([])
predicted_y = np.array([])

model.eval()
for batch in test_dataloader:
    x_test, y_test = batch
    output = model(x_test.float())
    loss = 0.5 * torch.norm(model.linear.weight.squeeze())**2
    loss += C * torch.clamp(1 - y_test * output, min =0).mean()
    
    total_loss += loss
    n_batch += 1
    
    true_y = np.append(true_y, y_test.numpy())
    pred_y = 2 *(output >= 0) -1 
    predicted_y = np.append(predicted_y, pred_y.numpy())

print(f"Loss : {total_loss / n_batch} \t Accuracy : {accuracy(true_y, predicted_y)}")    

Loss : 0.14827470481395721 	 Accuracy : 0.9298245614035088


# Sklearn SVM

In [19]:
x_train, x_test, y_train, y_test =train_test_split(data['data'], 2 * data['target'] -1,test_size= 0.2, random_state = seed )

In [20]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train, y_train)

SVC(kernel='linear')

In [21]:
accuracy(y_test, clf.predict(x_test))

0.956140350877193